In [1]:
from sqlalchemy import create_engine
import pandas as pd
from urllib.parse import quote_plus
from dotenv import load_dotenv
import os

In [2]:
load_dotenv("local.env")

True

In [3]:
olimpiadas_dados_caminho = "./dados/olimpiadas.csv"
df_olimpiadas = pd.read_csv(olimpiadas_dados_caminho, encoding="utf-8")
df_olimpiadas.columns = [column.lower() for column in df_olimpiadas.columns]
df_olimpiadas["id"] = df_olimpiadas["id"].astype(int)
df_olimpiadas = df_olimpiadas.drop_duplicates()
print(df_olimpiadas.shape)
df_olimpiadas.head(3)

(269731, 15)


,id,name,sex,age,height,weight,team,noc,games,year,season,city,sport,event,medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN


In [4]:
def get_ids_for_df_rows(df):
    df = df.astype(str)
    concatenated = df.apply(lambda row: ",".join(row.values), axis = 1)
    ids, uniques = pd.factorize(concatenated)
    return ids + 1

df = df_olimpiadas.copy(deep=True)

athlete_columns = ["name", "sex", "height", "weight"]
df["athlete_id"] = df["id"]
athlete_columns += ["athlete_id"]

noc_columns = ["noc"]
df["noc_id"] = get_ids_for_df_rows(df[noc_columns])
noc_columns += ["noc_id"]

team_columns = ["team"]
df["team_id"] =  get_ids_for_df_rows(df[team_columns])
team_columns += ["team_id"]

medal_columns = ["medal"]
df["medal_id"] =  get_ids_for_df_rows(df[medal_columns])
df["medal_id"] = df["medal_id"].replace(1, None)
df["medal_id"] = df["medal_id"] - 1
medal_columns += ["medal_id"]

city_columns = ["city"]
df["city_id"] =  get_ids_for_df_rows(df[city_columns])
city_columns += ["city_id"]

season_columns = ["season"]
df["season_id"] = get_ids_for_df_rows(df[season_columns])
season_columns += ["season_id"]

sport_columns = ["sport"]
df["sport_id"] = get_ids_for_df_rows(df[sport_columns])
sport_columns += ["sport_id"]

modality_columns = ["event", "sport_id"]
df["modality_id"] = get_ids_for_df_rows(df[modality_columns])
modality_columns += ["modality_id"]

games_columns = ["year", "season_id", "city_id"]
df["games_id"] = get_ids_for_df_rows(df[games_columns])
games_columns += ["games_id"]

event_columns = ["modality_id", "games_id"]
df["event_id"] = get_ids_for_df_rows(df[event_columns])
event_columns += ["event_id"]

athlete_event_columns = ["event_id", "athlete_id", "medal_id", "age", "noc_id", "team_id"]
df["athlete_event_id"] = get_ids_for_df_rows(df[athlete_event_columns])
athlete_event_columns += ["athlete_event_id"]

print(df.shape)
df.head()

(269731, 26)


,id,name,sex,age,height,weight,team,noc,games,year,...,noc_id,team_id,medal_id,city_id,season_id,sport_id,modality_id,games_id,event_id,athlete_event_id
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,...,1,1,NaN,1,1,1,1,1,1,1
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,...,1,1,NaN,2,1,2,2,2,2,2
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,...,2,2,NaN,3,1,3,3,3,3,3
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,...,2,3,1,4,1,4,4,4,4,4
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,...,3,4,NaN,5,2,5,5,5,5,5


In [5]:
athlete_df = df[athlete_columns]
athlete_df = athlete_df.drop_duplicates()
athlete_df.columns = ["name", "sex", "height", "weight", "id"]

noc_df = df[noc_columns]
noc_df = noc_df.drop_duplicates()
noc_df.columns = ["name", "id"]

team_df = df[team_columns]
team_df = team_df.drop_duplicates()
team_df.columns = ["name", "id"]

medal_df = df[medal_columns]
medal_df = medal_df.drop_duplicates()
medal_df = medal_df.dropna()
medal_df.columns = ["medal", "id"]

city_df = df[city_columns]
city_df = city_df.drop_duplicates()
city_df.columns = ["name", "id"]

season_df = df[season_columns]
season_df = season_df.drop_duplicates()
season_df.columns = ["name", "id"]

sport_df = df[sport_columns]
sport_df = sport_df.drop_duplicates()
sport_df.columns = ["name", "id"]

modality_df = df[modality_columns]
modality_df = modality_df.drop_duplicates()
modality_df.columns = ["name", "id_sport", "id"]

games_df = df[games_columns]
games_df = games_df.drop_duplicates()
games_df.columns = ["year", "id_season", "id_city", "id"]

event_df = df[event_columns]
event_df = event_df.drop_duplicates()
event_df.columns = ["id_modality", "id_games", "id"]

athlete_event_df = df[athlete_event_columns]
athlete_event_df = athlete_event_df.drop_duplicates()
athlete_event_df.columns = ["id_event", "id_athlete", "id_medal", "age", "id_noc", "id_team", "id"]

In [409]:
#Não esqueça de colocar os dados no local.env
username = os.getenv("DB_USERNAME")
password = quote_plus(os.getenv("DB_PASSWORD"))
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
database = os.getenv("DB_DATABASE")

In [410]:
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}', echo=True)

In [412]:
try:
    noc_df.to_sql('NOC', con=engine, if_exists='append', index=False)
    team_df.to_sql('Team', con=engine, if_exists='append', index=False)
    medal_df.to_sql('Medal', con=engine, if_exists='append', index=False)
    sport_df.to_sql('Sport', con=engine, if_exists='append', index=False)
    modality_df.to_sql('Modality', con=engine, if_exists='append', index=False)
    athlete_df.to_sql('Athlete', con=engine, if_exists='append', index=False)
    city_df.to_sql('City', con=engine, if_exists='append', index=False)
    season_df.to_sql('Season', con=engine, if_exists='append', index=False)
    games_df.to_sql('Games', con=engine, if_exists='append', index=False)
    event_df.to_sql('Event', con=engine, if_exists='append', index=False)
    athlete_event_df.to_sql('Athlete_Event', con=engine, if_exists='append', index=False)
    print("Dados inseridos com sucesso!")
except Exception as e:
    print("Ocorreu um erro durante a inserção dos dados:", e)

2024-06-24 23:41:19,606 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-06-24 23:41:19,607 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-06-24 23:41:19,609 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-06-24 23:41:19,609 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-06-24 23:41:19,609 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-06-24 23:41:19,609 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-06-24 23:41:19,609 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-24 23:41:19,609 INFO sqlalchemy.engine.Engine DESCRIBE `dados_olimpiada`.`NOC`
2024-06-24 23:41:19,625 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-06-24 23:41:19,641 INFO sqlalchemy.engine.Engine INSERT INTO `NOC` (name, id) VALUES (%(name)s, %(id)s)
2024-06-24 23:41:19,642 INFO sqlalchemy.engine.Engine [generated in 0.00305s] [{'name': 'CHN', 'id': 1}, {'name': 'DEN', 'id': 2}, {'name': 'NED', 'id': 3}, {'name': 'USA', 'id': 4}, {'name': 'FIN', 'id': 5}, {'name': 'NOR', 'id': 6}, 

C:\Users\scame\AppData\Local\Temp\ipykernel_14316\434737351.py:2: UserWarning: The provided table name 'NOC' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  noc_df.to_sql('NOC', con=engine, if_exists='append', index=False)
C:\Users\scame\AppData\Local\Temp\ipykernel_14316\434737351.py:3: UserWarning: The provided table name 'Team' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  team_df.to_sql('Team', con=engine, if_exists='append', index=False)
C:\Users\scame\AppData\Local\Temp\ipykernel_14316\434737351.py:4: UserWarning: The provided table name 'Medal' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  medal_df.to_sql('Medal', con=engine, if_exists='append', index=False)
C:\Users\s

2024-06-24 23:41:19,867 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-24 23:41:19,870 INFO sqlalchemy.engine.Engine DESCRIBE `dados_olimpiada`.`Modality`
2024-06-24 23:41:19,872 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-06-24 23:41:19,892 INFO sqlalchemy.engine.Engine INSERT INTO `Modality` (name, id_sport, id) VALUES (%(name)s, %(id_sport)s, %(id)s)
2024-06-24 23:41:19,901 INFO sqlalchemy.engine.Engine [generated in 0.01602s] [{'name': "Basketball Men's Basketball", 'id_sport': 1, 'id': 1}, {'name': "Judo Men's Extra-Lightweight", 'id_sport': 2, 'id': 2}, {'name': "Football Men's Football", 'id_sport': 3, 'id': 3}, {'name': "Tug-Of-War Men's Tug-Of-War", 'id_sport': 4, 'id': 4}, {'name': "Speed Skating Women's 500 metres", 'id_sport': 5, 'id': 5}, {'name': "Speed Skating Women's 1,000 metres", 'id_sport': 5, 'id': 6}, {'name': "Cross Country Skiing Men's 10 kilometres", 'id_sport': 6, 'id': 7}, {'name': "Cross Country Skiing Men's 50 kilometres", 'id_sport': 6, 'id': 8}

C:\Users\scame\AppData\Local\Temp\ipykernel_14316\434737351.py:6: UserWarning: The provided table name 'Modality' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  modality_df.to_sql('Modality', con=engine, if_exists='append', index=False)


2024-06-24 23:41:20,896 INFO sqlalchemy.engine.Engine INSERT INTO `Athlete` (name, sex, height, weight, id) VALUES (%(name)s, %(sex)s, %(height)s, %(weight)s, %(id)s)
2024-06-24 23:41:20,896 INFO sqlalchemy.engine.Engine [generated in 0.68429s] [{'name': 'A Dijiang', 'sex': 'M', 'height': 180.0, 'weight': 80.0, 'id': 1}, {'name': 'A Lamusi', 'sex': 'M', 'height': 170.0, 'weight': 60.0, 'id': 2}, {'name': 'Gunnar Nielsen Aaby', 'sex': 'M', 'height': None, 'weight': None, 'id': 3}, {'name': 'Edgar Lindenau Aabye', 'sex': 'M', 'height': None, 'weight': None, 'id': 4}, {'name': 'Christine Jacoba Aaftink', 'sex': 'F', 'height': 185.0, 'weight': 82.0, 'id': 5}, {'name': 'Per Knut Aaland', 'sex': 'M', 'height': 188.0, 'weight': 75.0, 'id': 6}, {'name': 'John Aalberg', 'sex': 'M', 'height': 183.0, 'weight': 72.0, 'id': 7}, {'name': 'Cornelia "Cor" Aalten (-Strannood)', 'sex': 'F', 'height': 168.0, 'weight': None, 'id': 8}  ... displaying 10 of 135571 total bound parameter sets ...  {'name': 'P

...

2024-06-24 23:41:28,601 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-24 23:41:28,601 INFO sqlalchemy.engine.Engine DESCRIBE `dados_olimpiada`.`Event`
2024-06-24 23:41:28,601 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-06-24 23:41:28,665 INFO sqlalchemy.engine.Engine INSERT INTO `Event` (id_modality, id_games, id) VALUES (%(id_modality)s, %(id_games)s, %(id)s)
2024-06-24 23:41:28,665 INFO sqlalchemy.engine.Engine [generated in 0.04633s] [{'id_modality': 1, 'id_games': 1, 'id': 1}, {'id_modality': 2, 'id_games': 2, 'id': 2}, {'id_modality': 3, 'id_games': 3, 'id': 3}, {'id_modality': 4, 'id_games': 4, 'id': 4}, {'id_modality': 5, 'id_games': 5, 'id': 5}, {'id_modality': 6, 'id_games': 5, 'id': 6}, {'id_modality': 5, 'id_games': 6, 'id': 7}, {'id_modality': 6, 'id_games': 6, 'id': 8}  ... displaying 10 of 6192 total bound parameter sets ...  {'id_modality': 765, 'id_games': 30, 'id': 6191}, {'id_modality': 689, 'id_games': 3, 'id': 6192}]
2024-06-24 23:41:29,215 INFO sqlalch

C:\Users\scame\AppData\Local\Temp\ipykernel_14316\434737351.py:11: UserWarning: The provided table name 'Event' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  event_df.to_sql('Event', con=engine, if_exists='append', index=False)


2024-06-24 23:41:29,415 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-24 23:41:29,441 INFO sqlalchemy.engine.Engine DESCRIBE `dados_olimpiada`.`Athlete_Event`
2024-06-24 23:41:29,446 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-06-24 23:41:32,468 INFO sqlalchemy.engine.Engine INSERT INTO `Athlete_Event` (id_event, id_athlete, id_medal, age, id_noc, id_team, id) VALUES (%(id_event)s, %(id_athlete)s, %(id_medal)s, %(age)s, %(id_noc)s, %(id_team)s, %(id)s)
2024-06-24 23:41:32,468 INFO sqlalchemy.engine.Engine [generated in 2.61476s] [{'id_event': 1, 'id_athlete': 1, 'id_medal': None, 'age': 24.0, 'id_noc': 1, 'id_team': 1, 'id': 1}, {'id_event': 2, 'id_athlete': 2, 'id_medal': None, 'age': 23.0, 'id_noc': 1, 'id_team': 1, 'id': 2}, {'id_event': 3, 'id_athlete': 3, 'id_medal': None, 'age': 24.0, 'id_noc': 2, 'id_team': 2, 'id': 3}, {'id_event': 4, 'id_athlete': 4, 'id_medal': 1, 'age': 34.0, 'id_noc': 2, 'id_team': 3, 'id': 4}, {'id_event': 5, 'id_athlete': 5, 'id_medal': None,

C:\Users\scame\AppData\Local\Temp\ipykernel_14316\434737351.py:12: UserWarning: The provided table name 'Athlete_Event' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  athlete_event_df.to_sql('Athlete_Event', con=engine, if_exists='append', index=False)


Dados inseridos com sucesso!


In [451]:
query_medalhas_pais = f"""
SELECT 
    n.name as country,
    m.medal as medal,
    COUNT(*) as total
FROM Athlete_Event ae
JOIN Medal m ON ae.id_medal = m.id
JOIN NOC n ON ae.id_noc = n.id
GROUP BY country, medal
ORDER BY total DESC
"""

df_medalhas_pais = pd.read_sql_query(query_medalhas_pais, engine)
df_medalhas_pais

2024-06-25 00:22:27,641 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-25 00:22:27,643 INFO sqlalchemy.engine.Engine 
SELECT 
    n.name as country,
    m.medal as medal,
    COUNT(*) as total
FROM Athlete_Event ae
JOIN Medal m ON ae.id_medal = m.id
JOIN NOC n ON ae.id_noc = n.id
GROUP BY country, medal
ORDER BY total DESC

2024-06-25 00:22:27,644 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-06-25 00:22:27,856 INFO sqlalchemy.engine.Engine ROLLBACK


,country,medal,total
0,USA,Gold,2638
1,USA,Silver,1641
2,USA,Bronze,1358
3,URS,Gold,1082
4,GER,Bronze,746
...,...,...,...
357,CRC,Silver,1
358,GHA,Silver,1
359,IOA,Silver,1
360,CIV,Silver,1


In [453]:
with open('./dados/medalhas_pais.json', "w", encoding="utf-8") as f:
    df_medalhas_pais.to_json(f, orient='records', indent = 4, force_ascii=False)

In [456]:
query_modalidades_maior_participacao = f"""
SELECT 
    m.name as modality,
    COUNT(*) as total
FROM Athlete_Event ae
JOIN Event e ON ae.id_event = e.id
JOIN Modality m ON e.id_modality = m.id
GROUP BY modality
ORDER BY total DESC
"""

df_modalidades_maior_participacao = pd.read_sql_query(query_modalidades_maior_participacao, engine)
df_modalidades_maior_participacao

2024-06-25 00:32:43,858 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-25 00:32:43,859 INFO sqlalchemy.engine.Engine 
SELECT 
    m.name as modality,
    COUNT(*) as total
FROM Athlete_Event ae
JOIN Event e ON ae.id_event = e.id
JOIN Modality m ON e.id_modality = m.id
GROUP BY modality
ORDER BY total DESC

2024-06-25 00:32:43,861 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-06-25 00:32:44,330 INFO sqlalchemy.engine.Engine ROLLBACK


,modality,total
0,Football Men's Football,5733
1,Ice Hockey Men's Ice Hockey,4762
2,Hockey Men's Hockey,3958
3,Water Polo Men's Water Polo,3358
4,Basketball Men's Basketball,3280
...,...,...
760,Basque Pelota Men's Two-Man Teams With Cesta,2
761,Sailing Mixed 18 foot,2
762,Archery Men's Championnat Du Monde,2
763,"Archery Men's Target Archery, 28 metres, Indiv...",2


In [457]:
with open('./dados/modalidades_maior_participacao.json', "w", encoding="utf-8") as f:
    df_modalidades_maior_participacao.to_json(f, orient='records', indent = 4, force_ascii=False)